# Set references for v1 tuning using the linear function

This notebook is dedicated to apply the linear correction in the neural network output w.r.t the avgmu. 

**NOTE**: This is an extra study.

**NOTE**: Get all models with 2 neurons.

In [1]:
from kolmov import crossval_table, get_color_fader, fit_table
import saphyra
import numpy as np
import pandas as pd
import collections
import os
import matplotlib
import matplotlib.pyplot as plt
from pprint import pprint
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

Welcome to JupyROOT 6.16/00
Using all sub packages with ROOT dependence


In [2]:
etbins = [0.0, 7.0, 10.0, 15.0]
etabins = [0.0, 0.8, 1.37, 1.54, 2.37, 2.50]

## 1) Reading all tunings:

Since I don't have v6 tuning files available, I will reload the production files and get the models for each bin.

In [3]:
def create_op_dict(op):
    d = {
              op+'_pd_ref'    : "reference/"+op+"_cutbased/pd_ref#0",
              op+'_fa_ref'    : "reference/"+op+"_cutbased/fa_ref#0",
              op+'_sp_ref'    : "reference/"+op+"_cutbased/sp_ref",
              op+'_pd_val'    : "reference/"+op+"_cutbased/pd_val#0",
              op+'_fa_val'    : "reference/"+op+"_cutbased/fa_val#0",
              op+'_sp_val'    : "reference/"+op+"_cutbased/sp_val",
              op+'_pd_op'     : "reference/"+op+"_cutbased/pd_op#0",
              op+'_fa_op'     : "reference/"+op+"_cutbased/fa_op#0",
              op+'_sp_op'     : "reference/"+op+"_cutbased/sp_op",

              # Counts
              op+'_pd_ref_passed'    : "reference/"+op+"_cutbased/pd_ref#1",
              op+'_fa_ref_passed'    : "reference/"+op+"_cutbased/fa_ref#1",
              op+'_pd_ref_total'     : "reference/"+op+"_cutbased/pd_ref#2",
              op+'_fa_ref_total'     : "reference/"+op+"_cutbased/fa_ref#2",
              op+'_pd_val_passed'    : "reference/"+op+"_cutbased/pd_val#1",
              op+'_fa_val_passed'    : "reference/"+op+"_cutbased/fa_val#1",
              op+'_pd_val_total'     : "reference/"+op+"_cutbased/pd_val#2",
              op+'_fa_val_total'     : "reference/"+op+"_cutbased/fa_val#2",
              op+'_pd_op_passed'     : "reference/"+op+"_cutbased/pd_op#1",
              op+'_fa_op_passed'     : "reference/"+op+"_cutbased/fa_op#1",
              op+'_pd_op_total'      : "reference/"+op+"_cutbased/pd_op#2",
              op+'_fa_op_total'      : "reference/"+op+"_cutbased/fa_op#2",
    }
    return d

tuned_info = collections.OrderedDict( {
              # validation
              "max_sp_val"      : 'summary/max_sp_val',
              "max_sp_pd_val"   : 'summary/max_sp_pd_val#0',
              "max_sp_fa_val"   : 'summary/max_sp_fa_val#0',
              # Operation
              "max_sp_op"       : 'summary/max_sp_op',
              "max_sp_pd_op"    : 'summary/max_sp_pd_op#0',
              "max_sp_fa_op"    : 'summary/max_sp_fa_op#0',
              } )

tuned_info.update(create_op_dict('tight'))
tuned_info.update(create_op_dict('medium'))
tuned_info.update(create_op_dict('loose'))
tuned_info.update(create_op_dict('vloose'))

In [4]:
analysis_path = os.environ['ANALYSIS_PATH']
file_name     = os.path.join(analysis_path, 'v1/r1/best_inits.csv')

In [5]:
cv_v1  = crossval_table(tuned_info, etbins = etbins , etabins = etabins )
cv_v1.from_csv(file_name)

In [6]:
best_inits_v1_r1 = cv_v1.filter_inits("max_sp_val")
best_inits_v1_r1.head()

,Unnamed: 0,train_tag,et_bin,eta_bin,model_idx,sort,init,file_name,tuned_idx,max_sp_val,...,vloose_pd_ref_total,vloose_fa_ref_total,vloose_pd_val_passed,vloose_fa_val_passed,vloose_pd_val_total,vloose_fa_val_total,vloose_pd_op_passed,vloose_fa_op_passed,vloose_pd_op_total,vloose_fa_op_total
80,2789,v1.r1.mlp10,0,0,8,0,8,/home/micael/Documents/NeuralRinger/cern_data/...,0,0.943985,...,59183,375287,5854,5707,5919,37528,58529,54589,59183,375287
81,2692,v1.r1.mlp10,0,0,8,1,2,/home/micael/Documents/NeuralRinger/cern_data/...,0,0.946490,...,59183,375287,5854,5338,5919,37528,58529,55314,59183,375287
82,2706,v1.r1.mlp10,0,0,8,2,5,/home/micael/Documents/NeuralRinger/cern_data/...,0,0.944360,...,59183,375287,5854,5667,5919,37528,58529,55880,59183,375287
83,2611,v1.r1.mlp10,0,0,8,3,5,/home/micael/Documents/NeuralRinger/cern_data/...,0,0.943679,...,59183,375287,5853,6336,5918,37529,58529,57640,59183,375287
84,2621,v1.r1.mlp10,0,0,8,4,4,/home/micael/Documents/NeuralRinger/cern_data/...,0,0.943823,...,59183,375287,5853,5852,5918,37529,58529,54826,59183,375287


In [7]:
best_inits_v1_r1.train_tag.unique()

array(['v1.r1.mlp10', 'v1.r1.mlp11', 'v1.r1.mlp12', 'v1.r1.mlp13',
       'v1.r1.mlp14', 'v1.r1.mlp15', 'v1.r1.mlp16', 'v1.r1.mlp17',
       'v1.r1.mlp18', 'v1.r1.mlp19', 'v1.r1.mlp2', 'v1.r1.mlp20',
       'v1.r1.mlp3', 'v1.r1.mlp4', 'v1.r1.mlp5', 'v1.r1.mlp6',
       'v1.r1.mlp7', 'v1.r1.mlp8', 'v1.r1.mlp9'], dtype=object)

## select models 

here I choosed 2 neurons because I can't find any strong evidence that suggest to use another topology

In [8]:
best_sorts_v1_r1 = cv_v1.filter_sorts( best_inits_v1_r1.loc[best_inits_v1_r1.train_tag=='v1.r1.mlp2'] , 'max_sp_op')
best_models = cv_v1.get_best_models(best_sorts_v1_r1, remove_last=True)

In [9]:
best_models[0][0].keys()

dict_keys(['model', 'etBin', 'etaBin', 'etBinIdx', 'etaBinIdx', 'history'])

## 2) Linear correction:

Here we will set all thresholds to operate as the same pd reference from cut-based using the pileup linear correction strategy. As the classifier efficiency has some "dependence" w.r.t the pileup we adopt the linear adjustment to "fix" the trigger efficiency. Here we will "fix" the neural network threshold w.r.t the pileup. 

### 2.1) Get all PD/Fas values:

Read all reference values from the storage.

In [10]:
m_epsilons = {
    'tight_cutbased' : 0.0, 
    'medium_cutbased': 0.0, 
    'loose_cutbased' : 0.0, 
    'vloose_cutbased': 0.0
}

In [11]:
def my_physics_filter(raw_data, signal_pid):
  '''
  This function will filter the signal data given a pid. The background always will be filtered by veto veryloose.

  Arguments:.
  - raw_data: the raw_data results from dict(np.load(DATA))
  - signal_pid: the pid to be used for filter the signal
  '''

  # create a pandas DataFrame
  temp_df = pd.DataFrame(data=raw_data['data'], columns=raw_data['features'])
  temp_df['target'] = raw_data['target']
  # filter signal and background
  sgn_df  = temp_df[(temp_df.target == 1) & (temp_df['el_lh%s' %(signal_pid)] == 1)]
  bkg_df  = temp_df[(temp_df.target != 1) & (temp_df['el_lhvloose'] != 1)]
  # returning the concatenate dataframe
  return pd.concat([sgn_df, bkg_df], ignore_index=True)
  

Today we don't filter the data at prometheus so I need to calculate 

1. create new refereces:

$P_D = \frac{\text{aceitos pelo cut-based (ringer) operando em }}{\text{aceitos pelo offline operando em}}$

2. run correction using the data that don't have pid on in.

3. run efficiency

In [12]:
# calculate all pd/fa from reference file
#ref_path = os.path.join(os.environ['DATA_PATH'],
#                        'data17_13TeV.AllPeriods.sgn.probes_lhmedium_EGAM2.bkg.VProbes_EGAM7.GRL_v97/references/')
#ref_path+= 'data17_13TeV.AllPeriods.sgn.probes_lhmedium_EGAM2.bkg.VProbes_EGAM7.GRL_v97_et{ET}_eta{ETA}.ref.pic.gz'

path = os.path.join(os.environ['DATA_PATH'],
                        'data17-18_13TeV.AllPeriods.sgn.probes_EGAM2.bkg.VProbes_EGAM7.GRL_v97/')
path+= 'data17-18_13TeV.AllPeriods.sgn.probes_EGAM2.bkg.VProbes_EGAM7.GRL_v97_et{ET}_eta{ETA}.npz'

ref_paths = [[ path.format(ET=et,ETA=eta) for eta in range(5)] for et in range(3) ]
ref_matrix = {
            'tight'  : [[ {} for eta in range(5)] for et in range(3)],
            'medium' : [[ {} for eta in range(5)] for et in range(3)],
            'loose'  : [[ {} for eta in range(5)] for et in range(3)],
            'vloose' : [[ {} for eta in range(5)] for et in range(3)], 
              }
references = {
                'tight_cutbased'  : 'T0HLTElectronT2CaloTight', 
                'medium_cutbased' : 'T0HLTElectronT2CaloMedium', 
                'loose_cutbased'  : 'T0HLTElectronT2CaloLoose', 
                'vloose_cutbased' : 'T0HLTElectronT2CaloVLoose'}

from saphyra.core import ReferenceReader
for et_bin in range(3):
    for eta_bin in range(5):
        for name, selector in references.items():
            #refObj = ReferenceReader().load(ref_paths[et_bin][eta_bin])
            offline_pid = name.split('_')[0]
            # open file
            refObj = my_physics_filter(raw_data=dict(np.load(ref_paths[et_bin][eta_bin])),
                                       signal_pid=offline_pid)

            # define the filters
            t2calo_accept      = refObj[selector] == 1
            egam2_filter       = refObj.target == 1
            egam7_filter       = refObj.target != 1
            sgn_offline_filter = refObj['el_lh%s' %(offline_pid)] == 1
            bkg_offline_filter = refObj.el_lhvloose != 1
            
            # calculate the pd
            det_prob  = (len(refObj[(egam2_filter) & (sgn_offline_filter) & (t2calo_accept)])/\
                         len(refObj[(egam2_filter) & (sgn_offline_filter)]))
            # calculate the fa
            fake_prob = (len(refObj[(egam7_filter) & (bkg_offline_filter) & (t2calo_accept)])/\
                         len(refObj[(egam7_filter) & (bkg_offline_filter)])) 
            
            ref_matrix[offline_pid][et_bin][eta_bin][name] = {'pd':det_prob, 'fa':fake_prob, 'pd_epsilon': m_epsilons[name]}

In [13]:
ref_matrix['tight'][2][0]

{'tight_cutbased': {'pd': 0.990168709794878,
  'fa': 0.24183134174136167,
  'pd_epsilon': 0.0}}

### 2.2) Create data generator:

Since each tuning models is fed by a different data organization, we need to create a generator to open the data file, prepare the matrix and apply some pre-processing (if needed).

In [14]:
def generator( path, signal_pid ):
    def norm1( data ):
        norms = np.abs( data.sum(axis=1) )
        norms[norms==0] = 1
        return data/norms[:,None]
    from Gaugi import load
    d = dict(np.load(path))

    temp_df           = pd.DataFrame(data=d['data'], columns=d['features'])
    temp_df['target'] = d['target']

    rings = ['L2Calo_ring_%i' %(iring) for iring in range(100)]


    # filter signal and background
    sgn_df  = temp_df[(temp_df.target == 1) & (temp_df['el_lh%s' %(signal_pid)] == 1)]
    bkg_df  = temp_df[(temp_df.target != 1) & (temp_df['el_lhvloose'] != 1)]
    
    full_df = pd.concat([sgn_df, bkg_df], ignore_index=True)
    
    # extract rings
    data_rings = norm1(full_df[rings].values)
    target     = full_df['target'].values
    avgmu      = full_df['avgmu'].values
    
    return [data_rings], target, avgmu

def pattern_gen_tight(path):
    return generator(path, signal_pid='tight')
def pattern_gen_medium(path):
    return generator(path, signal_pid='medium')
def pattern_gen_loose(path):
    return generator(path, signal_pid='loose')
def pattern_gen_vloose(path):
    return generator(path, signal_pid='vloose')

In [15]:
generator_dict = {
                'tight'  : pattern_gen_tight,
                'medium' : pattern_gen_medium,
                'loose'  : pattern_gen_loose,
                'vloose' : pattern_gen_vloose
                }

In [16]:
#path = os.path.join(os.environ['DATA_PATH'],
#                        'data17_13TeV.AllPeriods.sgn.probes_lhmedium_EGAM2.bkg.VProbes_EGAM7.GRL_v97/')
#path+= 'data17_13TeV.AllPeriods.sgn.probes_lhmedium_EGAM2.bkg.VProbes_EGAM7.GRL_v97_et{ET}_eta{ETA}.npz'


path = os.path.join(os.environ['DATA_PATH'],
                        'data17-18_13TeV.AllPeriods.sgn.probes_EGAM2.bkg.VProbes_EGAM7.GRL_v97/')
path+= 'data17-18_13TeV.AllPeriods.sgn.probes_EGAM2.bkg.VProbes_EGAM7.GRL_v97_et{ET}_eta{ETA}.npz'

paths = [[ path.format(ET=et,ETA=eta) for eta in range(5)] for et in range(3)]

# try to modified the resolution

In [17]:
barrel = np.arange(1.,24.5,1.).tolist() + np.arange(24.5,32.5,.5).tolist() + np.arange(32.5,40.5,1.).tolist() + np.arange(40.5,45.5,2.5).tolist() + [45.5,50.5,60.5,70.5]
longbarrel = np.arange(1.,24.5,4.).tolist() + np.arange(24.5,32.5,2.).tolist() + np.arange(32.5,40.5,4.).tolist() + np.arange(40.5,45.5,5.).tolist() + [45.5,70.5]
longbarrelhigherE = np.arange(1.,24.5,4.).tolist() + np.arange(24.5,32.5,4.).tolist() + np.arange(32.5,40.5,8.).tolist() + [40.5,70.5]
#longbarrelhigherE = longbarrel
crack =  np.arange(1.,40.5,10.).tolist() + [40.5,70.5]
endcap = longbarrel
endcaphigherE = crack
lastendcap = [1.,20.5,30.5,60.5,70.5]
  
res_ybins = [[ barrel, longbarrel, crack, endcap, lastendcap] for _ in range(3)]
res_ybins[1][0] = longbarrel
res_ybins[1][1] = longbarrelhigherE
res_ybins[2][0] = longbarrelhigherE
res_ybins[2][1] = longbarrelhigherE
res_ybins[2][3] = endcaphigherE
res_ybins[2][2] = crack

In [18]:
# create the table class
from ROOT import kBlackBody
aux_dict = {'tight'  : 'Tight',
            'medium' : 'Medium', 
            'loose'  : 'Loose',
            'vloose' : 'VeryLoose'
            }

ct = {}

model_name_format = 'data17-18_13TeV_EGAM2_probes_lhmedium_EGAM7_vetolhvloose.model_v1.electron{op}.et%d_eta%d'
config_name_format = 'ElectronRinger{op}TriggerConfig.conf'

for ikey in generator_dict.keys():
    # initialize
    print('init correction')
    print('\n -> %s' %(ikey))
    
    ct[ikey]  = fit_table(generator=generator_dict[ikey], etbins=etbins , etabins=etabins, 
                          x_bin_size=0.001, y_bin_size=res_ybins, ymin=0., ymax=70, 
                          xmin_percentage=0.05, xmax_percentage=99.95, palette=kBlackBody )

    print('\n -> filling')
    ct[ikey].fill(paths, best_models, ref_matrix[ikey], 
                  'correction_v1_r1_probes_lh%s_EGAM2_vetoProbes_EGAM7' %(ikey), 
                  except_these_bins = [(0,2),(0,4), (1,2),(1,4),(2,2),(2,4)])

    print('\n -> dumping beamer pdf')
    ct[ikey].dump_beamer_table(ct[ikey].table(), best_models, 'data17-18_13TeV v1_r1 tuning (Jpsiee)', 
                     'correction_v1_r1_data17-18_13TeV_EGAM2_probes_lh%s_EGAM7_vetoProbes.pdf' %(ikey))
    # export models
    print('\n -> exporting models')
    ct[ikey].export(best_models, 
                         model_name_format.format(op=aux_dict[ikey]), 
                         config_name_format.format(op=aux_dict[ikey]),
                         '%s_cutbased' %(ikey),
                         #references['%s_cutbased' %(op.lower())],
                         to_onnx=False)

re/texlive/texmf-dist/tex/latex/translator/translator-basic-dictionary
-English.dict)
(/usr/share/texlive/texmf-dist/tex/latex/translator/translator-numbers-dictiona
ry-English.dict))
(/usr/share/texlive/texmf-dist/tex/latex/translator/translator-theorem-dictiona
ry-English.dict
(/usr/share/texlive/texmf-dist/tex/latex/translator/translator-bibliography-dic
tionary-English.dict)
(/usr/share/texlive/texmf-dist/tex/latex/translator/translator-environment-dict
ionary-English.dict)
(/usr/share/texlive/texmf-dist/tex/latex/translator/translator-months-dictionar
y-English.dict))
(/usr/share/texlive/texmf-dist/tex/latex/translator/translator-numbers-dictiona
ry-English.dict)
(/usr/share/texlive/texmf-dist/tex/latex/translator/translator-theorem-dictiona
ry-English.dict) (/usr/share/texlive/texmf-dist/tex/latex/ucs/ucsencs.def) (/usr/share/texlive/texmf-dist/tex/latex/ucs/ucsencs.def)
No file correction_v1_r1_data17-18_13TeV_EGAM2_probes_lhloose_EGAM7_vetoProbes.
nav.
No file correction_v1_r1_

# default correction

In [17]:
# create the table class
from ROOT import kBlackBody
aux_dict = {'tight'  : 'Tight',
            'medium' : 'Medium', 
            'loose'  : 'Loose',
            'vloose' : 'VeryLoose'
            }

ct = {}

model_name_format = 'data17-18_13TeV_EGAM2_probes_lhmedium_EGAM7_vetolhvloose.model_v1.electron{op}.et%d_eta%d'
config_name_format = 'ElectronRinger{op}TriggerConfig.conf'

for ikey in generator_dict.keys():
    # initialize
    print('init correction')
    print('\n -> %s' %(ikey))
    ct[ikey]  = fit_table(generator_dict[ikey], etbins , etabins, 0.001, 1.5, 16.5, 45.5, 
                          xmin_percentage=0.05, xmax_percentage=99.95, palette=kBlackBody )

    print('\n -> filling')
    ct[ikey].fill(paths, best_models, ref_matrix[ikey], 
                  'correction_v1_r1_probes_lh%s_EGAM2_vetoProbes_EGAM7' %(ikey), 
                  except_these_bins = [(0,2),(0,4), (1,2),(1,4),(2,2),(2,4)])

    print('\n -> dumping beamer pdf')
    ct[ikey].dump_beamer_table(ct[ikey].table(), best_models, 'data17-18_13TeV v1_r1 tuning (Jpsiee)', 
                     'correction_v1_r1_data17-18_13TeV_EGAM2_probes_lh%s_EGAM7_vetoProbes.pdf' %(ikey))
    # export models
    print('\n -> exporting models')
    ct[ikey].export(best_models, 
                         model_name_format.format(op=aux_dict[ikey]), 
                         config_name_format.format(op=aux_dict[ikey]),
                         '%s_cutbased' %(ikey),
                         #references['%s_cutbased' %(op.lower())],
                         to_onnx=False)

mf-dist/tex/latex/beamer/beamerinnerthemerounded.sty)
(/usr/share/texlive/texmf-dist/tex/latex/beamer/beamerouterthemesmoothtree.sty)
(./correction_v1_r1_data17-18_13TeV_EGAM2_probes_lhloose_EGAM7_vetoProbes.aux)
*geometry* driver: auto-detecting
*geometry* detected driver: pdftex
(/usr/share/texlive/texmf-dist/tex/context/base/mkii/supp-pdf.mkii
[Loading MPS to PDF converter (version 2006.09.02).]
) (/usr/share/texlive/texmf-dist/tex/latex/epstopdf-pkg/epstopdf-base.sty
(/usr/share/texlive/texmf-dist/tex/latex/latexconfig/epstopdf-sys.cfg))
ABD: EveryShipout initializing macros
(/usr/share/texlive/texmf-dist/tex/latex/hyperref/nameref.sty
(/usr/share/texlive/texmf-dist/tex/latex/refcount/refcount.sty)
(/usr/share/texlive/texmf-dist/tex/generic/gettitlestring/gettitlestring.sty))
(./correction_v1_r1_data17-18_13TeV_EGAM2_probes_lhloose_EGAM7_vetoProbes.out)
(./correction_v1_r1_data17-18_13TeV_EGAM2_probes_lhloose_EGAM7_vetoProbes.out)
(/usr/share/texlive/texmf-dist/tex/latex/translator

## check if pile-up to 60

In [19]:
# create the table class
from ROOT import kBlackBody
aux_dict = {'tight'  : 'Tight',
            'medium' : 'Medium', 
            'loose'  : 'Loose',
            'vloose' : 'VeryLoose'
            }

ct = {}

model_name_format = 'data17-18_13TeV_EGAM2_probes_lhmedium_EGAM7_vetolhvloose.model_v1.electron{op}.et%d_eta%d'
config_name_format = 'ElectronRinger{op}TriggerConfig.conf'

for ikey in generator_dict.keys():
    # initialize
    print('init correction')
    print('\n -> %s' %(ikey))
    
    ct[ikey]  = fit_table(generator=generator_dict[ikey], etbins=etbins , etabins=etabins, 
                          x_bin_size=0.001, y_bin_size=1.5, ymin=10.5, ymax=60.5, 
                          xmin_percentage=0.05, xmax_percentage=99.95, palette=kBlackBody )

    print('\n -> filling')
    ct[ikey].fill(paths, best_models, ref_matrix[ikey], 
                  'correction_v1_r1_probes_lh%s_EGAM2_vetoProbes_EGAM7' %(ikey), 
                  except_these_bins = [(0,2),(0,4), (1,2),(1,4),(2,2),(2,4)])

    print('\n -> dumping beamer pdf')
    ct[ikey].dump_beamer_table(ct[ikey].table(), best_models, 'data17-18_13TeV v1_r1 tuning (Jpsiee)', 
                     'correction_v1_r1_data17-18_13TeV_EGAM2_probes_lh%s_EGAM7_vetoProbes.pdf' %(ikey))
    # export models
    print('\n -> exporting models')
    ct[ikey].export(best_models, 
                         model_name_format.format(op=aux_dict[ikey]), 
                         config_name_format.format(op=aux_dict[ikey]),
                         '%s_cutbased' %(ikey),
                         #references['%s_cutbased' %(op.lower())],
                         to_onnx=False)


(/usr/share/texlive/texmf-dist/tex/latex/refcount/refcount.sty)
(/usr/share/texlive/texmf-dist/tex/generic/gettitlestring/gettitlestring.sty
(/usr/share/texlive/texmf-dist/tex/latex/translator/translator-basic-dictionary
-English.dict)))
(/usr/share/texlive/texmf-dist/tex/latex/translator/translator-bibliography-dic
tionary-English.dict
(./correction_v1_r1_data17-18_13TeV_EGAM2_probes_lhloose_EGAM7_vetoProbes.out))
(./correction_v1_r1_data17-18_13TeV_EGAM2_probes_lhloose_EGAM7_vetoProbes.out)
(/usr/share/texlive/texmf-dist/tex/latex/translator/translator-environment-dict
ionary-English.dict)
(/usr/share/texlive/texmf-dist/tex/latex/translator/translator-months-dictionar
y-English.dict)
(/usr/share/texlive/texmf-dist/tex/latex/translator/translator-numbers-dictiona
ry-English.dict)
(/usr/share/texlive/texmf-dist/tex/latex/translator/translator-basic-dictionary
-English.dict
(/usr/share/texlive/texmf-dist/tex/latex/translator/translator-theorem-dictiona
ry-English.dict))
(/usr/share/tex

### 2.3) Apply linear correction:

**NOTE**: Take about 25 minutes.

In [23]:
ct.keys()

dict_keys(['tight', 'medium', 'loose', 'vloose'])

In [24]:
ct['tight']

In [ ]:
# Fill it
ct.fill(paths, best_models, ref_matrix,
        'correction_v1_probes_lhmedium_EGAM2_vetoProbes_EGAM7',
         except_these_bins = [(0,2),(0,4), (1,2),(1,4),(2,2),(2,4)])

In [ ]:
ct.table().head()

### 2.3) Create beamer report:

In [ ]:
ct.dump_beamer_table(ct.table(), best_models, 'data17_13TeV v1 tuning (Jpsiee)', 
                     'correction_v1_data17_13TeV_EGAM2_probes_lhmedium_EGAM7_vetoProbes.pdf')

## Export tunings:

In [26]:
'Tight'.lower()

'tight'

In [37]:
len(best_models[0])

5

In [38]:
import itertools
for iet, ieta in itertools.product(range(3), range(5)):
    print(iet, ieta, best_models[iet][ieta]['thresholds'], '\n')

0 0 {'vloose_cutbased': {'offset': -1.31045849489784, 'slope': -0.014082744232930365, 'threshold': -1.7369999999999999, 'reference_pd': 0.9823712812144876, 'reference_fa': 0.3373459377590863, 'figures': ['/home/micael/workspace/doc_research/repositories/ringer_analysis/2020/comissioning/correction_v1/correction_v1_probes_lhvloose_EGAM2_vetoProbes_EGAM7/th2_signal_vloose_cutbased_et0_eta0.pdf', '/home/micael/workspace/doc_research/repositories/ringer_analysis/2020/comissioning/correction_v1/correction_v1_probes_lhvloose_EGAM2_vetoProbes_EGAM7/th2_background_vloose_cutbased_et0_eta0.pdf']}} 

0 1 {'vloose_cutbased': {'offset': -1.6609877376194488, 'slope': -0.009032003945985894, 'threshold': -1.9357500000000003, 'reference_pd': 0.9735674843849009, 'reference_fa': 0.6800795166460057, 'figures': ['/home/micael/workspace/doc_research/repositories/ringer_analysis/2020/comissioning/correction_v1/correction_v1_probes_lhvloose_EGAM2_vetoProbes_EGAM7/th2_signal_vloose_cutbased_et0_eta1.pdf', '/h

In [28]:

model_name_format = 'data17_13TeV_EGAM2_probes_lhmedium_EGAM7_vetolhvloose.model_v1.electron{op}.et%d_eta%d'
config_name_format = 'ElectronRinger{op}TriggerConfig.conf'
for idx, op in enumerate(['Tight','Medium','Loose','VeryLoose']):
    ct[op.lower()].export(best_models, 
                         model_name_format.format(op=op), 
                         config_name_format.format(op=op),
                         '%s_cutbased' %(op.lower()),
                         #references['%s_cutbased' %(op.lower())],
                         to_onnx=False)

KeyError: 'tight_cutbased'